In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## DATA

In [2]:
dtype = {
    "Part": np.int8,
    "Site": np.int32,
    "p_A": np.float32,
    "p_C": np.float32,
    "p_G": np.float32,
    "p_T": np.float32,
}
states = pd.read_csv("../data/example_nematoda/anc.state", sep="\t", comment="#", dtype=dtype)
states = states.sort_values(["Node", "Part", "Site"])
print(states.shape)
states.head()

(1423266, 8)


,Node,Part,Site,State,p_A,p_C,p_G,p_T
1413243,Node1,1,1,A,0.56753,0.00222,0.02273,0.40751
1413244,Node1,1,2,T,0.00001,0.00003,0.00000,0.99996
1413245,Node1,1,3,T,0.17456,0.05253,0.16762,0.60528
1413246,Node1,1,4,T,0.10452,0.01489,0.07249,0.80810
1413247,Node1,1,5,T,0.00014,0.00067,0.00007,0.99912


## SCHEMES

In [3]:
def load_scheme(path: str):
    """
    parse files like scheme_birds_genes.nex (just separated genes)

    return dict(charset_lbl: gene_fp)
    """
    import re, os

    with open(path) as handle:
        raw_file = handle.read()
    charsets = re.findall("charset\s(\w+)\s?=\s?([\w_\.]+)(\s?:.+)?;", raw_file)
    scheme = {i: os.path.basename(fp) for i, (_, fp, _) in enumerate(charsets, 1)}
    return scheme

In [4]:
states.Part.value_counts()

5    417480
2    315666
3    280734
4    273066
1    136320
Name: Part, dtype: int64

In [5]:
iqtree_scheme = [
    ["ATP6", "ND6"], 
    ["COX1", "COX2"],
    ["COX3", "ND4"],
    ["CYTB", "ND1"],
    ["ND2", "ND3", "ND4L", "ND5"],
]
universal_scheme = load_scheme("../data/example_nematoda/scheme_devilworm.nex")
universal_scheme_rev = {y.split(".")[0]: x for x, y in universal_scheme.items()}
print(universal_scheme_rev)

{'ATP6': 1, 'COX1': 2, 'COX2': 3, 'COX3': 4, 'CYTB': 5, 'ND1': 6, 'ND2': 7, 'ND3': 8, 'ND4': 9, 'ND4L': 10, 'ND5': 11, 'ND6': 12}


## GENES LENGTH

In [6]:
import glob

gene_lens = dict()
for path in glob.glob("../data/example_nematoda/aln/*"):
    gname = path.split("/")[-1].split(".")[0]
    with open(path) as fin:
        fin.readline()
        glen = len(fin.readline().strip())
        gene_lens[gname] = glen

print(gene_lens)
# plus one
# ATP6,544
# COX1,1540
# COX2,685
# CYTB,1066
# COX3,766
# ND1,859
# ND2,820
# ND3,325
# ND4,1213
# ND4L,229
# ND6,418
# ND5,1570

{'COX2': 684, 'ND5': 1569, 'ND3': 324, 'COX3': 765, 'ATP6': 543, 'CYTB': 1065, 'COX1': 1539, 'ND2': 819, 'ND4L': 228, 'ND6': 417, 'ND1': 858, 'ND4': 1212}


## REPARTING

In [7]:
for part, genes in enumerate(iqtree_scheme, 1):
    print(part, genes, [gene_lens[g] for g in genes])
    part_len = ((states.Node == states.Node.sample().values[0]) & (states.Part == part)).sum()
    _gl = sum((gene_lens[g] for g in genes))
    assert part_len == _gl

    rf = states.Node.nunique()

    full_part_len = 0
    for g in genes:
        gl = gene_lens[g]
        sites = np.arange(1, gl + 1) + full_part_len
        
        where = (states.Part == part) & (states.Site.isin(sites)) # TODO
        states.loc[where, "NewPart"] = universal_scheme_rev[g]
        states.loc[where, "NewSite"] = np.tile(np.arange(1, gl + 1), rf)
        full_part_len += gl

1 ['ATP6', 'ND6'] [543, 417]
2 ['COX1', 'COX2'] [1539, 684]
3 ['COX3', 'ND4'] [765, 1212]
4 ['CYTB', 'ND1'] [1065, 858]
5 ['ND2', 'ND3', 'ND4L', 'ND5'] [819, 324, 228, 1569]


In [8]:
states["Part"] = states["NewPart"].astype(np.int8)
states["Site"] = states["NewSite"].astype(np.int32)
states.drop(["NewPart", "NewSite"], axis=1, inplace=True)

In [9]:
print(gene_lens)
print(universal_scheme_rev)

{'COX2': 684, 'ND5': 1569, 'ND3': 324, 'COX3': 765, 'ATP6': 543, 'CYTB': 1065, 'COX1': 1539, 'ND2': 819, 'ND4L': 228, 'ND6': 417, 'ND1': 858, 'ND4': 1212}
{'ATP6': 1, 'COX1': 2, 'COX2': 3, 'COX3': 4, 'CYTB': 5, 'ND1': 6, 'ND2': 7, 'ND3': 8, 'ND4': 9, 'ND4L': 10, 'ND5': 11, 'ND6': 12}


In [10]:
for part in universal_scheme:
    print(part, gene_lens[universal_scheme[part].split('.')[0]])

1 543
2 1539
3 684
4 765
5 1065
6 858
7 819
8 324
9 1212
10 228
11 1569
12 417


In [11]:
states.groupby("Part").Site.max()

Part
1      543
2     1539
3      684
4      765
5     1065
6      858
7      819
8      324
9     1212
10     228
11    1569
12     417
Name: Site, dtype: int32

In [73]:
states.to_csv("../data/example_nematoda/genes_states.tsv", sep='\t', index=None)